In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import torchvision
import torchvision.transforms as transforms

In [5]:
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [6]:
print(torch.__version__)
print(torchvision.__version__)

1.6.0
0.7.0


In [30]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


train_set = torchvision.datasets.FashionMNIST(
root='./data/FashionMNIST', train=True, download=True,transform=transforms.Compose([
    transforms.ToTensor()
])) 

In [63]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):

        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [65]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
batch = next(iter(train_loader))
images, labels = batch

In [66]:
preds = network(images)
loss = F.cross_entropy(preds, labels)
loss.item() #calculating the loss

2.3153624534606934

In [68]:
loss.backward()

In [72]:
optimizer = optim.Adam(network.parameters(),lr=0.01)

In [82]:
optimizer.step()